In [ ]:
import ast
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import norm, t, skewnorm, gaussian_kde
from plotly.subplots import make_subplots

def smart_global_format(x):
    if abs(x) < 10_000:
        return f"{x:,.4f}"
    else:
        return f"{x:,.0f}"

pd.options.display.float_format = smart_global_format


# Data cleaning

In [ ]:
# from google.colab import files

# uploaded = files.upload()
df = pd.read_csv("mm_bitget.csv")
df_futures = pd.read_csv("mm_bitget_futures.csv")
df_inventory = pd.read_csv("cake_inventory.csv")

# Convert timestamp to datetime
df["time"] = pd.to_datetime(df["createdTime"], unit="ms")
df_futures["time"] = pd.to_datetime(df_futures["requestTime"], unit="ms")
df_inventory["time"] = pd.to_datetime(df_inventory["time"], format="%Y-%m-%d %H:%M:%S,%f")

# Sort data by time ascending (oldest first)
df = df.sort_values(by="time", ascending=True).reset_index(drop=True)
df_futures = df_futures.sort_values(by="time", ascending=True).reset_index(drop=True)
df_inventory = df_inventory.sort_values(by="time", ascending=True).reset_index(drop=True)

# Set time as index
df = df.set_index("time")
df_futures = df_futures.set_index("time")
df_inventory = df_inventory.set_index("time")


def parse_fee(row):
    try:
        fee_info = ast.literal_eval(row["feeDetail"])[0]
        fee_coin = fee_info["feeCoin"]
        fee_value = float(fee_info["fee"])

        # Convert fee to USDT
        if fee_coin == "USDT":
            return fee_value  # already in USDT
        else:
            return fee_value * row["execPrice"]  # convert to USDT equivalent
    except:
        return 0.0

df["fee_usdt"] = df.apply(parse_fee, axis=1)

cols_to_drop = ["execId", "orderId", "clientOid", "tradeSide", "createdTime", "updatedTime", "execPnl", "execLinkId", "feeDetail"]
df = df.drop(columns=cols_to_drop, errors="ignore")
df_futures = df_futures.drop(columns="requestTime", errors="ignore")

# from_date = input("Enter start date (YYYY-MM-DD): ")
# to_date = input("Enter end date (YYYY-MM-DD): ")

# # # === Convert to datetime safely ===
# if from_date:
#     from_date = pd.to_datetime(from_date)
# else:
#     from_date = df.index.min()  # earliest date in data

# if to_date:
#     to_date = pd.to_datetime(to_date)
# else:
#     to_date = df.index.max()  # latest date in data

# Filter data by date range
# df = df.loc[(df.index >= from_date) & (df.index <= to_date)]

print(f"Total records: {len(df)} records")
print(f"Total futures records: {len(df_futures)} records")
print(f"Total inventory records: {len(df_inventory)} records")

# Inventory reconstruction

In [ ]:
df["side_num"] = np.where(df["side"] == "buy", 1, -1)
df["net_qty"] = df["side_num"] * df["execQty"]
df["inventory"] = df["net_qty"].cumsum()
inventory = abs(df["inventory"].min())
df["inventory"] = df["inventory"] - min(df["inventory"].min(), 0)

equity = 2300
quoting_weight = 0.4
avg_price = equity * quoting_weight / df["inventory"].head(10).median()
# --- Tính inventory_median (target động) ---
inventory_median = equity * quoting_weight / df["execPrice"]

# --- Tính độ lệch tương đối ---
deviation_ratio = (df["inventory"].abs() / inventory_median)

In [ ]:
data = df_inventory['iv_skew'].resample('1min').mean()
data = data[data.index > "2025-12-01"]
data = data.dropna()

mu, sigma = data.mean(), data.std()
x_pdf = np.linspace(data.min(), data.max(), 500)

# =======================
# 1. Fit Skew-Normal Distribution
# =======================
a_sn, loc_sn, scale_sn = skewnorm.fit(data)
pdf_skewnorm = skewnorm.pdf(x_pdf, a_sn, loc=loc_sn, scale=scale_sn)

# =======================
# 2. Kernel Density Estimation (KDE)
# =======================
kde = gaussian_kde(data)
pdf_kde = kde(x_pdf)

# =======================
# Scale all PDF curves to histogram scale
# =======================
bin_width = (data.max() - data.min()) / 50  # same bins as histogram
scale_factor = len(data) * bin_width

pdf_skewnorm_scaled = pdf_skewnorm * scale_factor
pdf_kde_scaled = pdf_kde * scale_factor

# =======================
# Plotting
# =======================
fig = go.Figure()

# Histogram
fig.add_trace(go.Histogram(
    x=data,
    nbinsx=50,
    marker_color='#1f77b4',
    opacity=0.65,
    name="Histogram"
))

# Skew-normal
fig.add_trace(go.Scatter(
    x=x_pdf,
    y=pdf_skewnorm_scaled,
    mode='lines',
    line=dict(width=2, color='purple'),
    name='Skew-Normal'
))

# KDE
fig.add_trace(go.Scatter(
    x=x_pdf,
    y=pdf_kde_scaled,
    mode='lines',
    line=dict(width=2, color='black', dash='dash'),
    name='KDE'
))

fig.update_layout(
    title='IV Skew Ratio Distribution – Skew-Normal vs KDE',
    xaxis=dict(title='IV Skew Ratio'),
    yaxis_title='Frequency',
    template='plotly_white',
    bargap=0.05
)

fig.show()

In [ ]:
# --- Tạo figure ---
fig = go.Figure()

# --- Trace 1: Exec Price ---
fig.add_trace(go.Scatter(
    x=df.index,
    y=df["execPrice"],
    mode="lines",
    name="Exec Price",
    line=dict(color="orange", width=2),
    yaxis="y1"
))

# --- Trace 2: Inventory ---
fig.add_trace(go.Scatter(
    x=df.index,
    y=df["inventory"],
    mode="lines",
    name="Inventory",
    line=dict(color="steelblue", width=2),
    yaxis="y2"
))

# --- Trace 3: Median động của Inventory ---

fig.add_trace(go.Scatter(
    x=df.index,
    y=inventory_median,
    mode="lines",
    name="Inventory Median",
    line=dict(color="royalblue", dash="dot", width=2),
    yaxis="y2"
))

# --- Đường 0 inventory ---
fig.add_hline(
    y=0,
    line_dash="dash",
    line_color="gray",
    opacity=0.6,
    name="Inventory = 0"
)

# --- Annotation hiển thị giá trị median cuối cùng ---
fig.add_annotation(
    x=df.index.max(),
    y=inventory_median.iloc[-1],
    xref="x",
    yref="y2",
    text=f"Median: {inventory_median.iloc[-1]:.2f}",
    showarrow=False,
    font=dict(color="royalblue", size=12),
    bgcolor="rgba(255,255,255,0.7)",
    xanchor="right",
    yanchor="bottom"
)

fig.add_annotation(
    x=df.index.max(),
    y=inventory_median.iloc[-1],
    xref="x",
    yref="y2",
    text=f"Median: {inventory_median.iloc[-1]:.2f}",
    showarrow=False,
    font=dict(color="royalblue", size=12),
    bgcolor="rgba(255,255,255,0.7)",
    xanchor="right",
    yanchor="bottom"
)

# --- Layout ---
fig.update_layout(
    title=dict(
        text="Exec Price vs Inventory Over Time (with Median)",
        font=dict(size=18, color="black")
    ),
    xaxis=dict(
        title="Time",
        showgrid=True,
        gridcolor="lightgray",
        rangeslider=dict(visible=True),
        type="date"
    ),
    yaxis=dict(
        autorange=True,
        fixedrange=False,
        title="Exec Price (USDT)",
        titlefont=dict(color="orange"),
        tickfont=dict(color="orange")
    ),
    yaxis2=dict(
        title="Inventory (Qty)",
        titlefont=dict(color="steelblue"),
        tickfont=dict(color="steelblue"),
        overlaying="y",
        side="right"
    ),
    legend=dict(
        x=0.01, y=0.99,
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="gray",
        borderwidth=0.5
    ),
    template="plotly_white",
    hovermode="x unified",
    width=1000,
    height=550
)

fig.show()

In [ ]:
# Filter Buy/Sell trades
df_buy = df[df["side"] == "buy"]
df_sell = df[df["side"] == "sell"]

# --- Tính VWAP cho mỗi phía ---
vwap_buy = np.sum(df_buy["execPrice"] * df_buy["execQty"]) / np.sum(df_buy["execQty"]) if len(df_buy) > 0 else np.nan
vwap_sell = np.sum(df_sell["execPrice"] * df_sell["execQty"]) / np.sum(df_sell["execQty"]) if len(df_sell) > 0 else np.nan

# --- Spread Capture ---
if not np.isnan(vwap_buy) and not np.isnan(vwap_sell):
    mid_price = (vwap_buy + vwap_sell) / 2
    spread_capture_pct = (vwap_sell - vwap_buy) / mid_price * 100
else:
    spread_capture_pct = np.nan

print(f"Average Spread Capture: {spread_capture_pct:.4f}%")
print(f"VWAP Buy: {vwap_buy:.4f}, VWAP Sell: {vwap_sell:.4f}")

# Tính theo ngày
daily_spread = (
    df.groupby([df.index.date, "side"])
    .apply(lambda x: np.sum(x["execPrice"] * x["execQty"]) / np.sum(x["execQty"]))
    .unstack()
)

daily_spread["mid"] = (daily_spread["buy"] + daily_spread["sell"]) / 2
daily_spread["spread_capture_%"] = (daily_spread["sell"] - daily_spread["buy"]) / daily_spread["mid"] * 100


# Cumulative VWAP theo thời gian
df["vwap"] = (df["execPrice"] * df["execQty"]).cumsum() / df["execQty"].cumsum()
df["vwap_1h"] = (
    (df["execPrice"] * df["execQty"])
    .rolling("1h")
    .sum()
    / df["execQty"].rolling("1h").sum()
)
df["vwap_1d"] = (
    (df["execPrice"] * df["execQty"])
    .rolling("1d")
    .sum()
    / df["execQty"].rolling("1d").sum()
)

daily_spread.tail(7)

In [ ]:
# df must contain: execPrice, tradeScope, fee_usdt, indexed by time
# Create maker flag
df["is_maker"] = (df["tradeScope"] == "maker").astype(int)

# Rolling maker ratio (adjust window as needed)
maker_ratio = df["is_maker"].rolling("1h").mean()

# --- Build Plotly figure ---
fig = go.Figure()

# Price line
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["execPrice"],
        name="Price",
        mode="lines",
        line=dict(width=1.5)
    )
)

# Maker ratio line (secondary axis)
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=maker_ratio,
        name="Maker Ratio",
        mode="lines",
        line=dict(width=1.3, dash="dot"),
        yaxis="y2"
    )
)

# Layout
fig.update_layout(
    title="Price + Maker Ratio",
    xaxis=dict(title="Time"),
    yaxis=dict(title="Price"),
    yaxis2=dict(
        title="Maker Ratio (0-1)",
        overlaying="y",
        side="right",
        showgrid=False,
        range=[0, 1]
    ),
    height=550,
    template="plotly_white"
)

fig.show()

In [ ]:

df["cum_fee"] = df["fee_usdt"].cumsum()
df["fee_rate"] = abs(df["fee_usdt"]) / df["execValue"]

# --- Daily fee summary ---
daily_fee = (
    df.groupby([df.index.date, "tradeScope"])["fee_usdt"]
    .sum()
    .unstack(fill_value=0)
)
daily_fee["total_fee"] = daily_fee.sum(axis=1)
daily_fee["make/take_ratio"] = daily_fee['maker'] / daily_fee['taker']

# --- Hourly fee summary ---
hourly_fee = (
    df.groupby([df.index.floor("h"), "tradeScope"])["fee_usdt"]
    .sum()
    .unstack(fill_value=0)
)
hourly_fee["total_fee"] = hourly_fee.sum(axis=1)
daily_fee["fee_sum"] = daily_fee["total_fee"].cumsum()

# Display summary tables
print("=== Daily Fee Summary ===")
daily_fee.tail(7)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["cum_fee"],
        mode="lines",
        line=dict(color="darkred", width=2),
        name="Cumulative Fee"
    )
)

fig.update_layout(
    title=dict(
        text="Cumulative Trading Fee Over Time",
        font=dict(size=18, color="black")
    ),
    xaxis_title="Time",
    yaxis_title="Fee (USDT)",
    template="plotly_white",
    hovermode="x unified",
    width=1000,
    height=500,
)

fig.show()


In [ ]:
# Khởi tạo các biến
realized_pnl = []

for _, row in df.iterrows():
    side = row['side']
    qty = row['execQty']
    price = row['execPrice']

    if side == 'buy':
        # Cập nhật giá vốn trung bình (Weighted Average)
        total_cost = avg_price * inventory + price * qty
        inventory += qty
        avg_price = total_cost / inventory if inventory > 0 else 0
        realized_pnl.append(0.0)

    elif side == 'sell':
        # Khi bán, tính PnL
        pnl = (price - avg_price) * qty
        realized_pnl.append(pnl)
        inventory -= qty

        # Nếu inventory về 0, reset giá vốn
        if inventory <= 0:
            avg_price = 0.0

# Gán kết quả vào DataFrame
df['realized_pnl'] = realized_pnl
# Cộng thêm phí (vì fee_usdt là phí USD)
df['realized_pnl_net'] = df['realized_pnl'] + df['fee_usdt']
# Cộng dồn để xem lũy kế
df['cum_pnl_net'] = df['realized_pnl_net'].cumsum()
# --- Merge asof ---
df_merged = pd.merge_asof(
    df.sort_values("time"),
    df_futures.sort_values("time"),
    on="time", direction="backward" )

df_merged[["unrealisedPnl", "totalFunding"]] = (
    df_merged[["unrealisedPnl", "totalFunding"]] .ffill() # forward fill từ hàng trước
    .fillna(0) # nếu vẫn NaN (ví dụ hàng đầu tiên) thì gán 0
    )

df_merged = df_merged.set_index('time')
df_merged["cum_pnl_net"] = (df_merged["cum_pnl_net"] + df_merged["unrealisedPnl"].ffill() + df_merged["totalFunding"].ffill())
df_merged['volume'] = df_merged['execValue'].cumsum()
df_merged['MME_cum'] = df_merged['cum_pnl_net'] / df_merged['volume'] * 100

cols_to_drop = ["category", "symbol", "side_num", "net_qty", "vwap", "vwap_1h",	"vwap_1d", "cum_fee", "fee_rate"]
df_merged = df_merged.drop(columns=cols_to_drop, errors="ignore")
df_merged

In [ ]:
# df_maker = df_merged[df_merged['tradeScope'] == 'taker']
# df_maker = df_maker[df_maker.index > "2025-11-26 07:40:00"]
# df_maker

In [ ]:
fig = go.Figure()

# Thêm trace cho volume (line chart)
fig.add_trace(go.Scatter(
    x=df_merged.index,
    y=df_merged['volume'],
    mode='lines',
    name='Cumulative Volume',
    line=dict(color='royalblue', width=2)
))

# Thiết lập layout
fig.update_layout(
    title='Cumulative Trade Volume Over Time',
    xaxis_title='Time',
    yaxis_title='Cumulative Volume',
    hovermode='x unified',
    template='plotly_dark',
    width=900,
    height=500
)

# Hiển thị biểu đồ
fig.show()

In [ ]:
# Tạo figure
fig = go.Figure()

# Thêm đường MME_cum theo thời gian
fig.add_trace(go.Scatter(
    x=df_merged.index,
    y=df_merged["MME_cum"],
    mode="lines",
    name="MME_cum (%)",
    line=dict(color="royalblue", width=2)
))

# Tuỳ chọn hiển thị
fig.update_layout(
    title="Cumulative Market Maker Efficiency (MME_cum)",
    xaxis_title="Time",
    yaxis_title="MME_cum (%)",
    template="plotly_dark",
    hovermode="x unified",
    font=dict(size=12),
    height=600
)

# Thêm đường trung vị MME_cum
fig.add_scatter(
    x=df_merged.index,
    y=df_merged["MME_cum"].expanding().median(),
    mode="lines",
    line=dict(color="orange", dash="dot"),
    name="Expanding Median (MME_cum)"
)

# ---- Đường ngưỡng lỗ MME = -0.04% ----
fig.add_hline(
    y=0.00,  # vì trục là %
    line_dash="dash",
    line_color="red",
    annotation_text="Loss Threshold (-0.00%)",
    annotation_position="bottom right"
)

# Hiển thị biểu đồ
fig.show()

In [ ]:
# Tính daily volume (sum per day)
daily_volume = df['execValue'].resample('1D').sum()

# Tính cumulative mean (từ đầu đến hiện tại)
daily_volume_cummean = daily_volume.expanding().mean()

# Tính rolling mean 30 ngày gần nhất
daily_volume_rolling_mean = daily_volume.rolling(window='30D').mean()

# Gộp tất cả vào cùng DataFrame
daily_volume_df = pd.DataFrame({
    'daily_volume_sum': daily_volume,
    'cumulative_mean': daily_volume_cummean,
    'rolling_mean_30d': daily_volume_rolling_mean
})

daily_volume_df.tail(7)

In [ ]:
df_inventory = df_inventory.resample('1min').mean()

# Create stacked area chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_inventory.index,
    y=df_inventory["usdvalue"],
    name="Base USD Value",
    mode="lines",
    stackgroup="one",  # this enables stacking
    line=dict(width=0.5),
    hovertemplate="Time: %{x}<br>USD Value: %{y:.2f}<extra></extra>"
))

fig.add_trace(go.Scatter(
    x=df_inventory.index,
    y=df_inventory["quote"],
    name="Quote USDT",
    mode="lines",
    stackgroup="one",
    line=dict(width=0.5),
    hovertemplate="Time: %{x}<br>Base: %{y:.2f}<extra></extra>"
))

fig.add_trace(go.Scatter(
    x=df_inventory.index,
    y=df_inventory["hedge"],
    name="Hedge",
    mode="lines",
    stackgroup="one",
    line=dict(width=0.5),
    hovertemplate="Time: %{x}<br>Hedge: %{y:.2f}<extra></extra>"
))

# Layout styling
fig.update_layout(
    title="Portfolio Composition Over Time",
    xaxis_title="Time",
    yaxis_title="Value (USDT)",
    hovermode="x unified",
    template="plotly_dark",
    legend=dict(title="Components", orientation="h", y=-0.2),
    height=600
)

fig.show()

In [ ]:
df_inventory.loc[df_inventory["equity"] > 2695, "equity"] = df_inventory["equity"] - 200

# Create line chart for equity
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_inventory.index,
    y=df_inventory["equity"],
    mode="lines+markers",
    name="Equity (USDT)",
    line=dict(color="#00CC96", width=2),
    marker=dict(size=4),
    hovertemplate="Time: %{x}<br>Equity: %{y:.2f} USDT<extra></extra>"
))

# Layout settings
fig.update_layout(
    title="Equity Over Time",
    xaxis_title="Time",
    yaxis_title="Equity (USDT)",
    template="plotly_dark",
    hovermode="x unified",
    height=500,
    legend=dict(orientation="h", y=-0.2)
)

fig.show()